# **Machine Learning Project**

In questa esercitazione metteremo assieme tutte le nozioni apprese dall'inizio del corso per risolvere un task specifico di machine learning.

### **Task: Semi-supervised classification**

Il task che vogliamo risolvere è un task di classificazione, caratterizzato però dal fatto che solo una piccola parte dei dati che disponiamo possiede le annotazioni (label). Questa condizione è nota come `Semi-supervised learning`. 

### **Dataset**

Il dataset che utilizzeremo sarà Fashion-MNIST, che contiene immagini di articoli di Zalando, composto da un training set di 60.000 campioni e test set con 10.000 campioni. Ogni campione è in scala di grigi e ha risoluzione 28x28. Il dataset è composto da 10 classi.


## **Pseudo-label**

Lo **pseudo-labeling** è una tecnica utilizzata nell'ambito del *semi-supervised learning*. L'idea di base è quella di generare etichette "artificiali" (pseudo-etichette) per i dati non etichettati (unlabeled, "U"), in modo da utilizzarle durante il training del modello. Per generare queste etichette ci sono diverse strategie: nel contesto di questa esercitazione utilizzeremo un algoritmo di clustering (k-means).

Ecco i passaggi generali del processo di pseudo-labeling:

1.  **Addestramento Iniziale**: Si addestra un algortimo di clustering sul set non etichettato (U) utilizzando un numero di cluster pari al numero di classi.
2.  **Predizione su Dati Etichettati**: Utilizziamo l'algortimo addestrato al punto 1 per clusterizzare i dati etichettati (L), assegnandoli quindi ai cluster che abbiamo trovato durante l' addestramento iniziale.
3.  **Mappare i cluster alle etichette**: Creiamo un mapping tra i cluster e le etichette, in modo da capire quale etichetta corrisponde allo specifico cluster. Per fare ciò assegniamo ad ogni cluster la vera label più frequente assegnata a quel cluster, sfruttando la funzione `mode`:

```Python
from scipy.stats import mode
import numpy as np

etichette_nel_cluster = np.array([0, 1, 1, 2, 1, 0, 1])

risultato_mode = mode(etichette_nel_cluster)

print(f"Oggetto ModeResult: {risultato_mode}") # Output: ModeResul(mode=1, count=4)
print(f"Etichetta più frequente (moda): {risultato_mode.mode}") # Output: (moda): 1

# etichetta più frequente come singolo numero:
etichetta_predominante = risultato_mode.mode
print(f"Etichetta predominante per questo cluster: {etichetta_predominante}") # Output: 1
```

4.  **Estrazione pseudo-label**: Alla fine, la classe più presente in un cluster diventa l' etichetta scelta per tutti i campioni assegnati a quel cluster. 


**Vantaggi**:
*   Permette di sfruttare la grande quantità di dati non etichettati, che altrimenti andrebbero sprecati.
*   Può migliorare significativamente le prestazioni del modello rispetto all'addestramento con i soli dati etichettati, specialmente quando questi ultimi sono scarsi.

In [1]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode # For majority voting
from sklearn.neural_network import MLPClassifier

2025-05-31 08:07:28.138331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748678848.433370      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748678848.549541      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Nomi delle classi per Fashion-MNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### `load_and_preprocess_data()`

In questa funzione dovrete:

* Scaricare il dataset.
* Riordinare casualmente i dati.
* Effettuare reshape.
* Scalare i valori dei pixel all' intervallo [0,1].
* Ridurre il numero di campioni a 10.000 per il train e 1.000 per il test.

La funzione dovrà ritornare nel seguente ordine:

1. Il training set ridotto.
2. Le etichette di train ridotte.
3. Il test set ridotto.
4. Le etichette di test ridotte.

In [8]:
def load_and_preprocess_data():
    """Carica e pre-processa il dataset Fashion-MNIST."""
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    


    # Riordina casualmente i dati
    indices = np.arange(x_train.shape[0])
    np.random.seed(0)
    np.random.shuffle(indices)

    x_train = x_train[indices]
    y_train = y_train[indices]

    # Reshape 
    x_train = x_train.reshape((x_train.shape[0], -1))  # da (N, 28, 28) a (N, 784)
    x_test = x_test.reshape((x_test.shape[0], -1))

    # Normalizza i valori dei pixel all'intervallo [0, 1]
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    # Riduci il numero di campioni
    x_train = x_train[:8000]
    y_train = y_train[:8000]
    x_test = x_test[:1000]
    y_test = y_test[:1000]

    # Stampa le dimensioni dei set
    print(f"x_train: {x_train.shape}, y_train: {y_train.shape}")
    print(f"x_test: {x_test.shape}, y_test: {y_test.shape}")

    return x_train, y_train, x_test, y_test

#x_train,y_train,x_test,y_test = load_and_preprocess_data();

x_train: (8000, 784), y_train: (8000,)
x_test: (1000, 784), y_test: (1000,)


### `apply_pca_and_scale`

In questa funzione dovrete:

* Scalare il training set e il test set.
* Applicare PCA con un numero di componenti specificato come parametro della funzione (o, equivalentemente, con una frazione desiderata della varianza espressa).
* Stampare il numero di componenti.
* Stampare la varianza espressa.

La funzione dovrà ritornare nel seguente ordine:

1. Il training set trasformato con PCA.
2. Il test set trasformato con PCA.

In [9]:
def apply_pca_and_scale(x_train, x_test, n_components):
    """Applica StandardScaler e PCA."""
    #Standardizza i dati passati come parametro
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    # Applica la PCA con n_components passato come parametro
    pca = PCA(n_components=n_components)
    x_train_pca = pca.fit_transform(x_train_scaled)
    x_test_pca = pca.transform(x_test_scaled)

    # 3. Stampa info
    print(f"Numero di componenti PCA: {pca.n_components_}")
    print(f"Varianza totale espressa: {np.sum(pca.explained_variance_ratio_):.4f}")

    return x_train_pca, x_test_pca

#apply_pca_and_scale(x_train,x_test,2)


Numero di componenti PCA: 2
Varianza totale espressa: 0.3655


(array([[ 18.408848 ,  -3.84784  ],
        [ 15.928936 ,  11.222203 ],
        [ 16.321404 ,  -1.6166565],
        ...,
        [-18.872774 ,  -1.2469021],
        [ -9.546667 ,  -9.544022 ],
        [ -4.5439444, -16.474163 ]], dtype=float32),
 array([[-16.65734  ,   7.5726004],
        [ 22.385258 ,   9.194903 ],
        [  3.5681877, -16.788254 ],
        ...,
        [ 18.039417 ,  14.866414 ],
        [-22.466608 ,  -2.9541507],
        [-17.223724 ,   1.2535549]], dtype=float32))

### `create_semi_supervised_split`

In questa funzione dovrete:

* Splittare il training set in due insiemi, etichettato (L) e non etichettato (U) utilizzando  `train_test_split` con:

`test_size`=`(1.0 - labeled_fraction)`

* Stampare la shape del set etichettato.
* Stampare la shape del set non etichettato.

La funzione deve ritornare nell seguente ordine:

1. Il set etichettato.
2. Le etichette del set etichettato.
3. Il set non etichettato.
4. Le etichette del set non etichettato. **N.B.** Queste etichette verranno utilizzate **SOLO** per valutare le pseudo-labels, non per l'addestramento.


In [12]:
def create_semi_supervised_split(x_train_pca, y_train, labeled_fraction):
    """Crea gli insiemi etichettato (L) e non etichettato (U)."""
    # Split in labeled (L) e unlabeled (U)
    x_label, x_unlabel, y_label, y_unlabel = train_test_split(
        x_train_pca, y_train,
        test_size=(1.0 - labeled_fraction),
        stratify=y_train,
        random_state=42
    )
    
    # Stampa le shape
    print(f"Set etichettato (L) - x: {x_label.shape}, y: {y_label.shape}")
    print(f"Set non etichettato (U) - x: {x_unlabel.shape}, y: {y_unlabel.shape}")

    return x_label, y_label, x_unlabel, y_unlabel

    

### `get_pseudo_labels`

In questa funzione dovrete:

* Istanziare un algoritmo di clustering (ad esempio, k-means).
* Addestrare e predire i clustering sul set non etichettato.
* Predire i clustering del set etichettato.
* Mappare i cluster ad un etichetta, utilizzando per ogni cluster l'etichetta più presente, estraibile utilizzando la funzione `mode` presentata sopra.
* Generare un array `pseudo_labels` assegnando a ogni campione del set non etichettato l'etichetta corrispondente al cluster a cui è stato assegnato.

La funzione deve ritornare:

1. L' array `pseudo_labels`.

In [23]:
def get_pseudo_labels(x_unlabeled_pca, x_labeled_pca, y_labeled, n_clusters):

    # 1. Istanziare algoritmo di clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    # 2. Allenare il modello di clustering con i dati non etichettati e salvare le assegnazioni ai cluster
    cluster_assignments_unlabeled = kmeans.fit_predict(x_unlabeled_pca)

    # 3. Calcolare l'assegnamento dei dati etichettati ai cluster
    cluster_assignments_labeled =  kmeans.predict(x_labeled_pca)

    # 4. Mappiamo i cluster alle label vere più frequenti
    cluster_to_true_label_map = {}
    
    for k_idx in range(n_clusters):
        # 4.1 Troviamo per ogni cluster le etichette vere dei campioni che vi appartengono.
        labels_in_cluster = y_labeled[cluster_assignments_labeled == k_idx]
        # 4.2 Troviamo l'etichetta più frequente per quel cluster.
        if len(labels_in_cluster) > 0:
            risultato_mode = mode(labels_in_cluster, keepdims=False)
            print(f"Cluster {k_idx} - Oggetto ModeResult: {risultato_mode}")
            print(f"Etichetta più frequente (moda): {risultato_mode.mode}")

            etichetta_predominante = risultato_mode.mode 
            print(f"Etichetta predominante per questo cluster {k_idx}: {etichetta_predominante}")

            cluster_to_true_label_map[k_idx] = etichetta_predominante
        # 4.3 Salviamo l'etichetta più frequente per quel cluster in cluster_to_true_label_map. 
        # In questo dizionario, la chiave è il cluster e il valore è l'etichetta vera più frequente.

    pseudo_labels_list = []

    for c_assign in cluster_assignments_unlabeled:

        if c_assign in cluster_to_true_label_map:
            # ....
             pseudo_labels_list.append(cluster_to_true_label_map[c_assign])
        else:
            # Se il cluster non ha una mappatura, possiamo assegnare un'etichetta di default o ignorarlo
            pseudo_labels_list.append(np.nan)
    
    # 5. Convertiamo la lista in un array numpy 
    pseudo_labels = np.array(pseudo_labels_list)
    
    return pseudo_labels 

### `train_and_evaluate_classifier`

In questa funzione dovrete:

* Rimuovere eventuali campioni con etichette NaN (potrebbero provenire da pseudo labels non mappate).
* Istanziare il modello utilizzando `model_class` come oggetto e `classifier_args` come argomenti. Esempio:

```Python
model_class = MLPClassifier
classifier_args = {'max_iter': 200, 'hidden_layer_sizes': (100, 50)}
model = model_class(**classifier_args)

# Equivalente a:
model = MLPClassifier(max_iter=200, hidden_layer_sizes=(100, 50))

```

* Allenare il modello sul training set a cui sono stati rimossi i campioni con etichette NaN.
* Calcolare l' accuracy.
* Stampare il `title`, che consiste nel titolo dell' esperimento eseguito. Questo perchè tale funzione verrà riutilizzata diverse volte per più set di dati. Un titolo ci permetterà di identificare quali risultati stiamo producendo.
* Stampare l' accuracy.
* Stampare il classification report.

La funzione deve ritornare:

1. Il modello.


In [13]:
def train_and_evaluate_classifier(model_class, classifier_args, x_train, y_train, x_test, y_test, title, class_names_list):
    # 1. Rimuove i campioni di training con etichette NaN
    valid_indices_train = ~np.isnan(y_train)  # Mask: True dove y_train NON è NaN
    x_train_clean = x_train[valid_indices_train]
    y_train_clean = y_train[valid_indices_train].astype(int)  # Convertiamo da float a int

    # 2. Istanziare il modello con gli argomenti forniti
    model = model_class(**classifier_args)

    # 3. Allenare il modello
    model.fit(x_train_clean, y_train_clean)

    # 4. Fare predizioni sul test set
    y_pred = model.predict(x_test)

    # 5. Calcolare e stampare i risultati
    print(f"\n--- {title} ---")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=class_names_list))

    return model


### `main`

In questa funzione dovrete:

* Utilizzare la funzione `load_and_preprocess_data` per caricare e pre-processare i dati.
* Utilizzare la funzione `apply_pca_and_scale` per applicare scaling e PCA.
* Utilizzare la funzione `create_semi_supervised_split` per dividere il train set in set etichettato e non etichettato.
* Utilizzare la funzione `get_pseudo_labels` per calcoalre le pseudo labels sul set non etichettato.
* Calcolare l' accuracy delle pseudo labels, cioè confrontarle con quelle vere in modo da vedere quanto sono accurate.
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello solo sui dati etichettati (L).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello solo sui dati non etichettati (U).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello sui dati etichettati (L) più quelli non etichettati (U).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello su tutto il dataset originale.

In [17]:
def main(classifier_class, classifier_args, n_components_pca, labeled_fraction, n_clusters):
    # 1. Caricamento e preprocessing dati
    x_train, y_train, x_test, y_test = load_and_preprocess_data()

    # 2. Scaling e PCA
    x_train_pca, x_test_pca = apply_pca_and_scale(x_train, x_test, n_components_pca)

    # 3. Split semi-supervised in labeled (L) e unlabeled (U)
    x_labeled, y_labeled, x_unlabeled, y_unlabeled = create_semi_supervised_split(
        x_train_pca, y_train, labeled_fraction)

    # 4. Calcolo pseudo labels sul set non etichettato
    pseudo_labels = get_pseudo_labels(x_unlabeled, x_labeled, y_labeled, n_clusters)

    # 5. Accuracy pseudo-labels (confronto con vere etichette del set non etichettato)
    valid_mask = ~np.isnan(pseudo_labels)
    pseudo_acc = np.mean(pseudo_labels[valid_mask] == y_unlabeled[valid_mask])
    print(f"\nAccuracy pseudo-labels (su set non etichettato): {pseudo_acc:.4f}")

    # 6. Allenamento e valutazione solo sul set etichettato (L)
    train_and_evaluate_classifier(
        classifier_class, classifier_args,
        x_labeled, y_labeled,
        x_test_pca, y_test,
        title="Solo dati etichettati (L)",
        class_names_list=class_names)

    # 7. Allenamento e valutazione solo sul set non etichettato (U) con pseudo-labels
    train_and_evaluate_classifier(
        classifier_class, classifier_args,
        x_unlabeled, pseudo_labels,
        x_test_pca, y_test,
        title="Solo dati non etichettati (U) con pseudo-labels",
        class_names_list=class_names)

    # 8. Allenamento e valutazione su L + U con pseudo-labels
    x_combined = np.vstack((x_labeled, x_unlabeled))
    y_combined = np.concatenate((y_labeled, pseudo_labels))
    train_and_evaluate_classifier(
        classifier_class, classifier_args,
        x_combined, y_combined,
        x_test_pca, y_test,
        title="Dati etichettati (L) + non etichettati (U) con pseudo-labels",
        class_names_list=class_names)

    # 9. Allenamento e valutazione su tutto il dataset originale (train + test set unito)
    # Se vuoi solo train + test originale, senza semi-supervisione:
    train_and_evaluate_classifier(
        classifier_class, classifier_args,
        x_train_pca, y_train,
        x_test_pca, y_test,
        title="Tutto il dataset originale",
        class_names_list=class_names)
    

### **Utilizzare la funzione `main`**

Specifichiamo adesso un set di parametri richiesti dalla funzione main e utilizziamola. Nello specifico la funzione main ha bisogno di:

* `classifier_class`: quale classificatore utilizzare, ad esempio `'MLPClassifier'`, `'LogisticRegression'` o altri visti in precedenza.
* `classifier_args`: un dizionario contenente i parametri del classificatore scelto, ad esempio un `MLPClassifier` necessiterà del parametro `hidden_layer_sizes`. Dipendentemente da quale classificatore scegliete dovrete creare il dizionario.
* `n_components_pca`: numero di componenti di PCA che vogliamo utilizzare. Se specifichiamo un valore compreso in [0, 1] questo verrà considerato come la percentuale di varianza che vogliamo mentenere.
* `labeled_fraction`: percentuale di dati da usare come insieme etichettato. Si consiglia il valore 0.002 corrispondente allo 0.2%, cioè 16 immagini su 8000.
* `n_clusters`: numero di cluster da utilizzare, nel nostro caso vogliamo che ci sia un cluster per ogni classe, quindi 10.

Infine utilizziamo la funzione main.

In [18]:
# Parametri
CLASSIFIER_CLASS = MLPClassifier  # Modello da usare, ad esempio LogisticRegression o SVC
CLASSIFIER_ARGS = {
    'max_iter': 20,
    'hidden_layer_sizes': (200,200)  # Aumenta il numero di iterazioni per la convergenza
}
N_COMPONENTS_PCA = 0.95  # Mantiene il 95% della varianza spiegata, o un numero fisso es. 50
LABELED_FRACTION = 0.002   # Frazione di dati da usare come insieme etichettato L
N_CLUSTERS = 10          # Fashion-MNIST ha 10 classi

In [24]:
main(
    CLASSIFIER_CLASS,
    CLASSIFIER_ARGS,
    N_COMPONENTS_PCA,
    LABELED_FRACTION,
    N_CLUSTERS
)

x_train: (8000, 784), y_train: (8000,)
x_test: (1000, 784), y_test: (1000,)
Numero di componenti PCA: 242
Varianza totale espressa: 0.9501
Set etichettato (L) - x: (16, 242), y: (16,)
Set non etichettato (U) - x: (7984, 242), y: (7984,)


/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 1 - Oggetto ModeResult: ModeResult(mode=1, count=2)
Etichetta più frequente (moda): 1
Etichetta predominante per questo cluster 1: 1
Cluster 2 - Oggetto ModeResult: ModeResult(mode=5, count=1)
Etichetta più frequente (moda): 5
Etichetta predominante per questo cluster 2: 5
Cluster 3 - Oggetto ModeResult: ModeResult(mode=4, count=1)
Etichetta più frequente (moda): 4
Etichetta predominante per questo cluster 3: 4
Cluster 4 - Oggetto ModeResult: ModeResult(mode=5, count=1)
Etichetta più frequente (moda): 5
Etichetta predominante per questo cluster 4: 5
Cluster 5 - Oggetto ModeResult: ModeResult(mode=2, count=2)
Etichetta più frequente (moda): 2
Etichetta predominante per questo cluster 5: 2
Cluster 7 - Oggetto ModeResult: ModeResult(mode=3, count=1)
Etichetta più frequente (moda): 3
Etichetta predominante per questo cluster 7: 3
Cluster 8 - Oggetto ModeResult: ModeResult(mode=9, count=1)
Etichetta più frequente (moda): 9
Etichetta predominante per questo cluster 8: 9
Cluster 9 - O

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and b


--- Solo dati non etichettati (U) con pseudo-labels ---
Accuracy: 0.3860
Classification Report:
              precision    recall  f1-score   support

 T-shirt/top       0.00      0.00      0.00       107
     Trouser       0.62      0.90      0.74       105
    Pullover       0.21      0.29      0.25       111
       Dress       0.18      0.26      0.21        93
        Coat       0.34      0.62      0.44       115
      Sandal       0.34      0.90      0.49        87
       Shirt       0.00      0.00      0.00        97
     Sneaker       0.00      0.00      0.00        95
         Bag       0.00      0.00      0.00        95
  Ankle boot       0.69      0.91      0.79        95

    accuracy                           0.39      1000
   macro avg       0.24      0.39      0.29      1000
weighted avg       0.24      0.39      0.29      1000



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and b


--- Dati etichettati (L) + non etichettati (U) con pseudo-labels ---
Accuracy: 0.3900
Classification Report:
              precision    recall  f1-score   support

 T-shirt/top       0.00      0.00      0.00       107
     Trouser       0.62      0.90      0.74       105
    Pullover       0.22      0.29      0.25       111
       Dress       0.19      0.28      0.23        93
        Coat       0.35      0.63      0.45       115
      Sandal       0.33      0.90      0.49        87
       Shirt       0.00      0.00      0.00        97
     Sneaker       0.00      0.00      0.00        95
         Bag       0.00      0.00      0.00        95
  Ankle boot       0.70      0.91      0.79        95

    accuracy                           0.39      1000
   macro avg       0.24      0.39      0.29      1000
weighted avg       0.24      0.39      0.30      1000


--- Tutto il dataset originale ---
Accuracy: 0.8400
Classification Report:
              precision    recall  f1-score   support



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
